In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.shape

(9557, 143)

In [5]:
df.columns[df.isnull().sum() > 0]

Index(['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'], dtype='object')

In [6]:
df.fillna(method='bfill',inplace=True)

In [7]:
df.columns[df.isnull().sum() > 0]

Index(['v18q1', 'rez_esc'], dtype='object')

In [8]:
df.fillna(method='ffill',inplace=True)

In [9]:
df.columns[df.isnull().sum() > 0]

Index([], dtype='object')

In [10]:

Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))

print(categorical_col)

{'edjefa', 'idhogar', 'Id', 'edjefe', 'dependency'}


In [11]:
df['edjefe'] = df['edjefe'].replace({'no': 0, 'yes':1}).astype(float)
df['edjefa'] = df['edjefa'].replace({'no': 0, 'yes':1}).astype(float)

In [12]:

Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))

print(categorical_col)

{'idhogar', 'Id', 'dependency'}


In [13]:
df['dependency'] = np.sqrt(df['SQBdependency'])

In [14]:
df.isna().sum().sum()

0

In [15]:
col_drops = ['Id','idhogar']
df.drop(col_drops,axis=1,inplace=True)

In [16]:
# finding numeric and categorical columns
Numeric_columns=df.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df.columns).difference(set(Numeric_columns))
# numeric cols to numeric
print(categorical_col)

set()


In [17]:
train_set = df

In [80]:
df['rent_per_adult'] = df['v2a1']/df['hogar_adul']
df['rent_per_person'] = df['v2a1']/df['hhsize']

df['overcrowding_room_and_bedroom'] = (df['hacdor'] + df['hacapo'])/2

df['no_appliances'] = df['refrig'] + df['computer'] + df['television']

# df1['home_condition_poor'] = (df1['epared1'] + df1['etecho1'] + df1['eviv1'])/3
# df1['home_condition_regular'] = (df1['epared2'] + df1['etecho2'] + df1['eviv2'])/3
# df1['home_condition_good'] = (df1['epared3'] + df1['etecho3'] + df1['eviv3'])/3

df['r4h1_percent_in_male'] = df['r4h1'] / df['r4h3']
df['r4m1_percent_in_female'] = df['r4m1'] / df['r4m3']
df['r4h1_percent_in_total'] = df['r4h1'] / df['hhsize']
df['r4m1_percent_in_total'] = df['r4m1'] / df['hhsize']
df['r4t1_percent_in_total'] = df['r4t1'] / df['hhsize']

df['adult'] = df['hogar_adul'] - df['hogar_mayor']
df['dependency_count'] = train_set['hogar_nin'] + df['hogar_mayor']
df['dependency'] = df['dependency_count'] / df['adult']
df['child_percent'] = df['hogar_nin']/df['hogar_total']
df['elder_percent'] = df['hogar_mayor']/df['hogar_total']
df['adult_percent'] = df['hogar_adul']/df['hogar_total']

df['rent_per_bedroom'] = df['v2a1']/df['bedrooms']
df['adults_per_bedroom'] = df['adult']/df['bedrooms']
df['child_per_bedroom'] = df['hogar_nin']/df['bedrooms']
df['male_per_bedroom'] = df['r4h3']/df['bedrooms']

df['female_per_bedroom'] = df['r4m3']/df['bedrooms']
df['bedrooms_per_person_household'] = df['hhsize']/df['bedrooms']

df['tablet_per_person_household'] = df['v18q1']/df['hhsize']
df['phone_per_person_household'] = df['qmobilephone']/df['hhsize']

df['age_12_19'] = df['hogar_nin'] - df['r4t1']

df['rent_per_room'] = df['v2a1']/df['rooms']
df['bedroom_per_room'] = df['bedrooms']/df['rooms']
df['elder_per_room'] = df['hogar_mayor']/df['rooms']
df['adults_per_room'] = df['adult']/df['rooms']
df['child_per_room'] = df['hogar_nin']/df['rooms']
df['male_per_room'] = df['r4h3']/df['rooms']
df['female_per_room'] = df['r4m3']/df['rooms']
df['room_per_person_household'] = df['hhsize']/df['rooms']
df['escolari_age'] = df['escolari']/df['age']

df['rez_esc_escolari'] = df['rez_esc']/df['escolari']
df['rez_esc_r4t1'] = df['rez_esc']/df['r4t1']
df['rez_esc_r4t2'] = df['rez_esc']/df['r4t2']
df['rez_esc_r4t3'] = df['rez_esc']/df['r4t3']
df['rez_esc_age'] = df['rez_esc']/df['age']


In [81]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB


In [21]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [108]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [46]:
!pip install bayesian-optimization

In [93]:
x_1 = df.drop('Target',axis=1)
y = df['Target']

In [103]:
cols= [col for col in x_1.columns if x_1[col].isnull().any()]
reduced_x_1 = x_1.drop(cols, axis=1)

# reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [104]:
df.shape

(9557, 178)

In [105]:
x_1.shape

(9557, 177)

In [106]:
reduced_x_1.shape

(9557, 171)

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate,cross_val_score

In [252]:
def optimize_lgb(data, targets):
    def lgb_crossval(n_estimators, num_leaves, min_child_samples, subsample):
        return lgb_cv(
            n_estimators=int(n_estimators),
            num_leaves = int(num_leaves),
            min_child_samples=int(min_child_samples),
            subsample=subsample,
            data=data,
            targets=targets,
        )

    optimizer = BayesianOptimization(
        f=lgb_crossval,
        pbounds={
            "n_estimators": (100,500),
            "num_leaves": (30,80),
            "min_child_samples": (5,30),
            "subsample": (0.6,1.0)
        },
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=25, init_points=20)

    print("Final result:", optimizer.max)

In [ ]:
print("--- Optimizing Light GBM ---")
optimize_lgb(x_1, y)

In [108]:

Lgbm=LGBMClassifier(n_estimators=200,learning_rate=0.1,random_state=45,num_leaves=200)
model = GaussianNB()
    
# model = RandomForestRegressor(n_estimators=100, random_state=0)

In [109]:
from sklearn.model_selection import train_test_split


In [110]:
x_train,x_test,y_train,y_test = train_test_split(reduced_x_1,y,random_state=42)
#x_train, x_valid, y_train, y_valid = train_test_split(x_1, y, train_size=0.8, test_size=0.2,random_state=0)

In [111]:
#Lgbm.fit(x_train,y_train)
model.fit(x_train,y_train)
# model.fit(x_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
Lgbm.score(x_test,y_test)


0.9435146443514645

In [35]:
df1 = pd.read_csv('test.csv')

In [36]:
df1.columns[df1.isnull().sum()> 0]

Index(['v2a1', 'v18q1', 'rez_esc', 'meaneduc', 'SQBmeaned'], dtype='object')

In [37]:
df1.fillna(method='bfill',inplace=True)

In [38]:
df1.columns[df1.isnull().sum()> 0]

Index(['v2a1', 'v18q1', 'rez_esc'], dtype='object')

In [39]:
df1.fillna(method='ffill', inplace=True)

In [40]:
df1.columns[df1.isnull().sum()> 0]

Index([], dtype='object')

In [41]:
Numeric_columns=df1.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df1.columns).difference(set(Numeric_columns))

print(categorical_col)

{'edjefa', 'idhogar', 'Id', 'edjefe', 'dependency'}


In [42]:
df1['edjefa'] = df1['edjefa'].replace({'no':0,'yes':1}).astype(float)
df1['edjefe'] = df1['edjefe'].replace({'no':0,'yes':1}).astype(float)


In [43]:
Numeric_columns=df1.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df1.columns).difference(set(Numeric_columns))

print(categorical_col)

{'idhogar', 'Id', 'dependency'}


In [44]:
df1['dependency'] = np.sqrt(df1['SQBdependency'])

In [45]:
Numeric_columns=df1.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df1.columns).difference(set(Numeric_columns))

print(categorical_col)

{'idhogar', 'Id'}


In [46]:
col_drops = ['Id','idhogar']
df1.drop(col_drops,inplace=True,axis=1)

In [47]:
Numeric_columns=df1.select_dtypes(include=np.number).columns.tolist()
categorical_col=set(df1.columns).difference(set(Numeric_columns))

print(categorical_col)

set()


In [48]:
test_set = df1

In [71]:
df1['rent_per_adult'] = df1['v2a1']/df1['hogar_adul']
df1['rent_per_person'] = df1['v2a1']/df1['hhsize']

df1['overcrowding_room_and_bedroom'] = (df1['hacdor'] + df1['hacapo'])/2

df1['no_appliances'] = df1['refrig'] + df1['computer'] + df1['television']

 
df1['r4h1_percent_in_male'] = df1['r4h1'] / df1['r4h3']
df1['r4m1_percent_in_female'] = df1['r4m1'] / df1['r4m3']
df1['r4h1_percent_in_total'] = df1['r4h1'] / df1['hhsize']
df1['r4m1_percent_in_total'] = df1['r4m1'] / df1['hhsize']
df1['r4t1_percent_in_total'] = df1['r4t1'] / df1['hhsize']

df1['adult'] = df1['hogar_adul'] - df1['hogar_mayor']
df1['dependency_count'] = df1['hogar_nin'] + df1['hogar_mayor']
df1['dependency'] = df1['dependency_count'] / df1['adult']
df1['child_percent'] = df1['hogar_nin']/df1['hogar_total']
df1['elder_percent'] = df1['hogar_mayor']/df1['hogar_total']
df1['adult_percent'] = df1['hogar_adul']/df1['hogar_total']

df1['rent_per_bedroom'] = df1['v2a1']/df1['bedrooms']
df1['adults_per_bedroom'] = df1['adult']/df1['bedrooms']
df1['child_per_bedroom'] = df1['hogar_nin']/df1['bedrooms']
df1['male_per_bedroom'] = df1['r4h3']/df1['bedrooms']

df1['female_per_bedroom'] = df1['r4m3']/df1['bedrooms']
df1['bedrooms_per_person_household'] = df1['hhsize']/df1['bedrooms']

df1['tablet_per_person_household'] = df1['v18q1']/df1['hhsize']
df1['phone_per_person_household'] = df1['qmobilephone']/df1['hhsize']

df1['age_12_19'] = df1['hogar_nin'] - df1['r4t1']

df1['rent_per_room'] = df1['v2a1']/df1['rooms']
df1['bedroom_per_room'] = df1['bedrooms']/df1['rooms']
df1['elder_per_room'] = df1['hogar_mayor']/df1['rooms']
df1['adults_per_room'] = df1['adult']/df1['rooms']
df1['child_per_room'] = df1['hogar_nin']/df1['rooms']
df1['male_per_room'] = df1['r4h3']/df1['rooms']
df1['female_per_room'] = df1['r4m3']/df1['rooms']
df1['room_per_person_household'] = df1['hhsize']/df1['rooms']
df1['escolari_age'] = df1['escolari']/df1['age']

df1['rez_esc_escolari'] = df1['rez_esc']/df1['escolari']
df1['rez_esc_r4t1'] = df1['rez_esc']/df1['r4t1']
df1['rez_esc_r4t2'] = df1['rez_esc']/df1['r4t2']
df1['rez_esc_r4t3'] = df1['rez_esc']/df1['r4t3']
df1['rez_esc_age'] = df1['rez_esc']/df1['age']

In [72]:
x1 = test_set

In [73]:
x1.shape

(23856, 177)

In [74]:
y.shape


(9557,)

In [75]:
x1.shape

(23856, 177)

In [76]:
Lgbm1=LGBMClassifier(n_estimators=200,learning_rate=0.1,random_state=45,num_leaves=500)


In [77]:
x1_train,x1_test,y1_train,y1_test = train_test_split(x1[:9557],y,random_state=45)

In [78]:
Lgbm1.fit(x1_train,y1_train)

LGBMClassifier(n_estimators=200, num_leaves=500, random_state=45)

In [79]:
Lgbm1.score(x1_test,y1_test)

0.7108786610878661